In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import os
import numpy as np
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [2]:
class HandPoseClassifier(nn.Module):
    def __init__(self):
        super(HandPoseClassifier, self).__init__()
        self.fc1 = nn.Linear(63, 128)  # Input layer to first hidden layer
        self.dropout1 = nn.Dropout(0.25)  # Dropout layer after the first hidden layer
        self.fc2 = nn.Linear(128, 128)  # Second hidden layer
        self.dropout2 = nn.Dropout(0.25)  # Dropout layer after the second hidden layer
        self.fc3 = nn.Linear(128, 64)   # Third hidden layer
        self.dropout3 = nn.Dropout(0.25)  # Dropout layer after the third hidden layer
        self.fc4 = nn.Linear(64, 7)     # Output layer

    def forward(self, x):
        x = F.relu(self.fc1(x))  # Activation function for the first hidden layer
        x = self.dropout1(x)     # Apply dropout
        x = F.relu(self.fc2(x))  # Activation function for the second hidden layer
        x = self.dropout2(x)     # Apply dropout
        x = F.relu(self.fc3(x))  # Activation function for the third hidden layer
        x = self.dropout3(x)     # Apply dropout
        x = self.fc4(x)          # No activation function here for raw scores to go into the loss function
        return x

In [3]:
data_files = os.listdir("./data/")

class_indexes = []
classes = []
X = []
y = []

for fil in data_files:
    parts = fil.split('_')
    idx = int(parts[0])
    class_name = parts[1][:-4]
    classes.append(class_name)
    class_indexes.append(idx)
    df = pd.read_csv("./data/" + fil)
    data = df.iloc[:].values
    one_hot = [0] * 7
    one_hot[idx] = 1
    for row in data:
        X.append(row)
        y.append(idx)

In [4]:
X_train, X_test_spare, y_train, y_test_spare = train_test_split(X, y, test_size=0.2, shuffle = True)
X_test, X_val, y_test, y_val = train_test_split(X_test_spare, y_test_spare, test_size=0.5, shuffle=True)

X_train = torch.from_numpy(np.array(X_train).astype(np.float32))
y_train = torch.from_numpy(np.array(y_train).astype(np.int64))

X_test = torch.from_numpy(np.array(X_test).astype(np.float32))
y_test = torch.from_numpy(np.array(y_test).astype(np.int64))

X_val = torch.from_numpy(np.array(X_val).astype(np.float32))
y_val = torch.from_numpy(np.array(y_val).astype(np.int64))

In [5]:
def get_batches(X, y, batch_size):
    num_samples = X.shape[0]
    for i in range(0, num_samples, batch_size):
        # Yield a tuple of the current batch's features and labels
        yield X[i:i + batch_size], y[i:i + batch_size]

In [9]:
model = HandPoseClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 200
batch_size = 8
print(model)

HandPoseClassifier(
  (fc1): Linear(in_features=63, out_features=128, bias=True)
  (dropout1): Dropout(p=0.25, inplace=False)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (dropout2): Dropout(p=0.25, inplace=False)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (dropout3): Dropout(p=0.25, inplace=False)
  (fc4): Linear(in_features=64, out_features=7, bias=True)
)


In [10]:
def train_model(model, X_train, y_train, X_val, y_val, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()

        # Validation loss
        model.eval()
        with torch.no_grad():
            val_output = model(X_val)
            val_loss = criterion(val_output, y_val)
        
        # Print loss every epoch or less frequently
        print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}')

In [11]:
train_model(model, X_train, y_train, X_val, y_val, criterion, optimizer, num_epochs)

Epoch 1/200, Training Loss: 1.947021245956421, Validation Loss: 1.9452214241027832
Epoch 2/200, Training Loss: 1.9441438913345337, Validation Loss: 1.940927505493164
Epoch 3/200, Training Loss: 1.9397190809249878, Validation Loss: 1.9364709854125977
Epoch 4/200, Training Loss: 1.9371174573898315, Validation Loss: 1.9316926002502441
Epoch 5/200, Training Loss: 1.9329721927642822, Validation Loss: 1.9262927770614624
Epoch 6/200, Training Loss: 1.9286588430404663, Validation Loss: 1.920225739479065
Epoch 7/200, Training Loss: 1.9248194694519043, Validation Loss: 1.913846492767334
Epoch 8/200, Training Loss: 1.9190200567245483, Validation Loss: 1.906894326210022
Epoch 9/200, Training Loss: 1.9133179187774658, Validation Loss: 1.899235486984253
Epoch 10/200, Training Loss: 1.9077582359313965, Validation Loss: 1.890954613685608
Epoch 11/200, Training Loss: 1.9018474817276, Validation Loss: 1.8818790912628174
Epoch 12/200, Training Loss: 1.893531084060669, Validation Loss: 1.8717892169952393


In [12]:
torch.save(model.state_dict(), "model.torch")